In [3]:
import numpy as np
import gurobipy as grb
import scipy.stats as stat
from tqdm.notebook import trange
import matplotlib.pyplot as plt


In [109]:
N = 100  # Fixed number of samples per replication
M = 10  # Number of replications
Nprim = 300  # Number of samples for upper bound calculation
alpha = 0.05

LB_seq = np.zeros(M)
LB_ci_up_seq = np.zeros(M)
LB_ci_low_seq = np.zeros(M)
LB_ci = np.zeros(M)

UB_seq = np.zeros(M)
UB_ci_up_seq = np.zeros(M)
UB_ci_low_seq = np.zeros(M)
UB_ci = np.zeros(M)

x_id = [1, 2]
y_id = [1, 2]

for rep_m in trange(M):
    # lower bound ###############################################################################
    w1 = np.random.uniform(1/2, 5, N)
    w2 = np.random.uniform(1/6, 3, N)

    LB_exp = 0  # each replication

    # model
    LB_model = grb.Model('SAA_LB')
    LB_model.setParam('OutputFlag', 0)

    # variables
    x = {}
    for k in x_id:
        x[k] = LB_model.addVar(lb=0, name="x(%s)" % (k))
    y = {}
    for k in y_id:
        for j in range(N):
            y[k, j] = LB_model.addVar(lb=0, name="y(%s_%s)" % (k, j))
    LB_model.update()

    # Define Objective
    obj_expr = 2 * x[1] + x[2] + grb.quicksum( y[1, k] +  y[2, k] for k in range(N)) * 1.0 / N
    LB_model.setObjective(obj_expr, grb.GRB.MINIMIZE)
    
    for k in range(N):
        LB_model.addConstr(y[1, k] >= 7 - w1[k] * x[1] - x[2], name="constr1(%s)" % (k))
        LB_model.addConstr(y[2, k] >= 4 - w2[k] * x[1] - x[2], name="constr2(%s)" % (k))
        
    LB_model.optimize()
    LB_exp = LB_model.objVal
    LB_seq[rep_m] = LB_exp

    # Upper Bound #######################################################################
    UB_exp = 0

    # model
    UB_model = grb.Model('SAA_UB')
    UB_model.setParam('OutputFlag', 0)

    # variables
    y = {}
    for k in y_id:
        for j in range(Nprim):
            y[k, j] = UB_model.addVar(lb=0, name="y(%s_%s)" % (k, j))
    UB_model.update()

    # Use optimal solution from previous replication for x1 and x2
    x_optimal = {1: LB_model.getVarByName("x(1)").X, 2: LB_model.getVarByName("x(2)").X}

    # Define Objective
    obj_expr = 2*x_optimal[1] + x_optimal[2] + grb.quicksum(y[1, kk] + y[2, kk] for kk in range(Nprim)) * 1.0 / Nprim
    UB_model.setObjective(obj_expr, grb.GRB.MINIMIZE)

    # Constraints
    for kk in range(min(Nprim, len(w1))):
        UB_model.addConstr(y[1, kk] >= 7 - w1[kk] * x_optimal[1] - x_optimal[2], name="constr1(%s)" % (kk))
        UB_model.addConstr(y[2, kk] >= 4 - w2[kk] * x_optimal[1] - x_optimal[2], name="constr2(%s)" % (kk))

    UB_model.optimize()
    UB_exp = UB_model.objVal
    UB_seq[rep_m] = UB_exp

# Calculate means and variances
LB_mean = np.mean(LB_seq)
LB_var = np.var(LB_seq, ddof=1)
UB_mean = np.mean(UB_seq)
UB_var = np.var(UB_seq, ddof=1)

# Calculate confidence intervals
LB_me = stat.norm.ppf(1-alpha)*np.sqrt(LB_var)/np.sqrt(M)
LB_ci_up = LB_mean + LB_me
LB_ci_low = LB_mean - LB_me

UB_me = stat    
    

  0%|          | 0/10 [00:00<?, ?it/s]

In [101]:
print("Lower Bound for each replication:")
for i, lb in enumerate(LB_seq):
    print(f"Replication {i+1}: {lb}")

Lower Bound for each replication:
Replication 1: 6.404796838732819
Replication 2: 6.509980445866116
Replication 3: 6.404435926997104
Replication 4: 6.691345176746547
Replication 5: 6.317875893001351
Replication 6: 6.255448587093016
Replication 7: 6.615614166802757
Replication 8: 6.337386474804037
Replication 9: 6.43984035321561
Replication 10: 6.5308805615893


In [103]:
print("Upper Bounds for each replication:")
for i, ub in enumerate(UB_seq):
    print(f"Replication {i+1}: {ub}")

Upper Bounds for each replication:
Replication 1: 5.595254336203238
Replication 2: 5.575114955724524
Replication 3: 5.504509634561117
Replication 4: 5.726683261402913
Replication 5: 5.482974379801021
Replication 6: 5.527386715885615
Replication 7: 5.710919283629896
Replication 8: 5.49649971036577
Replication 9: 5.493195312775013
Replication 10: 5.63802801331524


In [111]:
# Calculate average lower bound
avg_lower_bound = np.mean(LB_seq)

# Calculate standard error of the mean (SEM)
sem_lower_bound = stats.sem(LB_seq)

# Calculate confidence interval
alpha = 0.05  # significance level
ci_lower_bound = stats.norm.interval(1 - alpha, loc=avg_lower_bound, scale=sem_lower_bound)[0]
ci_upper_bound = stats.norm.interval(1 - alpha, loc=avg_lower_bound, scale=sem_lower_bound)[1]

# Print results
print("Average Lower Bound:", avg_lower_bound)
print("Confidence Interval for Lower Bound:", (ci_lower_bound, ci_upper_bound))


Average Lower Bound: 6.487106659556693
Confidence Interval for Lower Bound: (6.385183610251505, 6.58902970886188)


In [112]:
# Calculate average upper bound
avg_upper_bound = np.mean(UB_seq)

# Calculate standard error of the mean (SEM) for upper bound
sem_upper_bound = stats.sem(UB_seq)

# Calculate confidence interval for upper bound
alpha = 0.05  # significance level
ci_lower_bound_ub, ci_upper_bound_ub = stats.norm.interval(1 - alpha, loc=avg_upper_bound, scale=sem_upper_bound)

# Print results
print("Average Upper Bound:", avg_upper_bound)
print("Confidence Interval for Upper Bound:", (ci_lower_bound_ub, ci_upper_bound_ub))


Average Upper Bound: 5.601386823400315
Confidence Interval for Upper Bound: (5.538039596382962, 5.6647340504176675)


In [113]:
# Calculate optimality gap for each upper bound
optimality_gaps = [avg_lower_bound - ub for ub in UB_seq]

# Print optimality gaps
for rep_m, optimality_gap in enumerate(optimality_gaps):
    print(f"Optimality Gap for Replication {rep_m + 1}: {optimality_gap}")


Optimality Gap for Replication 1: 1.0063292205196728
Optimality Gap for Replication 2: 0.9331520236360644
Optimality Gap for Replication 3: 0.8016454512152018
Optimality Gap for Replication 4: 1.0205740890463844
Optimality Gap for Replication 5: 0.8671849792663853
Optimality Gap for Replication 6: 0.8417672726372993
Optimality Gap for Replication 7: 0.8396353203741018
Optimality Gap for Replication 8: 0.9451267357574071
Optimality Gap for Replication 9: 0.922640073787111
Optimality Gap for Replication 10: 0.6791431953241505


In [110]:
# Lower Bound #######################################################################
for rep_m in range(M):
    # Code to setup and optimize LB_model
    LB_model.optimize()

    # Check optimization status
    if LB_model.status == grb.GRB.Status.OPTIMAL:
        print(f"\nLower Bound Model - Replication {rep_m + 1}: Optimal solution found.")
        # Print objective value
        print("Objective value:", LB_model.objVal)
        # Print optimal solution values for variables
        print("Optimal solution values for variables:")
        for var in LB_model.getVars():
            print(f"{var.VarName}: {var.X}")
    else:
        print(f"\nLower Bound Model - Replication {rep_m + 1}: Optimization ended with status", LB_model.status)

# Upper Bound #######################################################################
for rep_m in range(M):
    # Code to setup and optimize UB_model
    UB_model.optimize()

    # Check optimization status
    if UB_model.status == grb.GRB.Status.OPTIMAL:
        print(f"\nUpper Bound Model - Replication {rep_m + 1}: Optimal solution found.")
        # Print objective value
        print("Objective value:", UB_model.objVal)
        # Print optimal solution values for variables
        print("Optimal solution values for variables:")
        for var in UB_model.getVars():
            print(f"{var.VarName}: {var.X}")
    else:
        print(f"\nUpper Bound Model - Replication {rep_m + 1}: Optimization ended with status", UB_model.status)



Lower Bound Model - Replication 1: Optimal solution found.
Objective value: 6.741540428889503
Optimal solution values for variables:
x(1): 0.836374243779997
x(2): 3.6684264943440676
y(1_0): 0.0
y(1_1): 1.5836398194755072
y(1_2): 1.2972039578947716
y(1_3): 0.21303854394260746
y(1_4): 2.907090206895529
y(1_5): 0.0
y(1_6): 0.4562927158972667
y(1_7): 2.08020407852791
y(1_8): 1.840188540057969
y(1_9): 2.858464318583098
y(1_10): 2.22094053668522
y(1_11): 0.24972081540178248
y(1_12): 1.9262649465885313
y(1_13): 0.10249537442133771
y(1_14): 1.6625123083741333
y(1_15): 2.0226640824660485
y(1_16): 0.0
y(1_17): 2.168021719245886
y(1_18): 1.4908280052091727
y(1_19): 1.6739603837949244
y(1_20): 1.1086069245279577
y(1_21): 1.0497543682415391
y(1_22): 1.6944264607723523
y(1_23): 1.9308402592928426
y(1_24): 0.6537529532517081
y(1_25): 2.0281633050041648
y(1_26): 0.15032852329694224
y(1_27): 2.805059608423399
y(1_28): 1.1383097561232272
y(1_29): 2.6954315107626186
y(1_30): 0.8880322143240247
y(1_31): 